<a href="https://colab.research.google.com/github/zihangdai/xlnet/blob/master/notebooks/colab_imdb_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
!echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!sudo apt-get update
!sudo apt-get install gcsfuse

deb http://packages.cloud.google.com/apt  main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   659  100   659    0     0  29954      0 --:--:-- --:--:-- --:--:-- 29954
OK
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.


In [67]:
!apt-get install gcsfuse

E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.
E: Malformed entry 1 in list file /etc/apt/sources.list.d/gcsfuse.list (Component)
E: The list of sources could not be read.


In [0]:
!sudo mkdir /content/GCS
!sudo chmod 777 /content/GCS

In [59]:
!gsutil ls

gs://xlnet_save_model/


In [68]:
!gcsfuse xlnet_save_model /content/GCS

/bin/bash: gcsfuse: command not found


# XLNet IMDB movie review classification project

This notebook is for classifying the [imdb sentiment dataset](https://ai.stanford.edu/~amaas/data/sentiment/).  It will be easy to edit this notebook in order to run all of the classification tasks referenced in the [XLNet paper](https://arxiv.org/abs/1906.08237). Whilst you cannot expect to obtain the state-of-the-art results in the paper on a GPU, this model will still score very highly. 

from google.colab import auth

auth.authenticate_user()

Once this is done you need to set your project and bucket name and then update the gcloud config.
project_id = [project_name]
bucket_name = [bucket_name]
!gcloud config set project {project_id}

After this has been done files can simply and quickly be upload or downloaded from the bucket with the following simple commands:

# download
!gsutil cp gs://{bucket_name}/foo.bar ./foo.bar

# upload
!gsutil cp  ./foo.bar gs://{bucket_name}/foo.bar

In [50]:
project_id = 'endless-bolt-246721'
bucket_name ='xlnet_save_model'
!gcloud config set project {project_id}

Updated property [core/project].


In [53]:
!gsutil cp -r /content/xlnet_cased_L-24_H-1024_A-16 gs://xlnet_save_model/xlnet_cased_L-24_H-1024_A-16

Copying file:///content/xlnet_cased_L-24_H-1024_A-16/spiece.model [Content-Type=application/octet-stream]...
Copying file:///content/xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.meta [Content-Type=application/octet-stream]...
Copying file:///content/xlnet_cased_L-24_H-1024_A-16/xlnet_config.json [Content-Type=application/json]...
Copying file:///content/xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index [Content-Type=application/octet-stream]...
\ [4 files][  2.0 MiB/  2.0 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significant

In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[your Cloud Platform project ID]'

In [45]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

# Generate a random bucket name to which we'll upload the file.
import uuid
bucket_name = 'colab-sample-bucket' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

W0714 21:53:49.435050 140203918296960 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

Done


In [0]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')
print('https://console.cloud.google.com/storage/browser?project={}'.format(project_id))

In [0]:
from google.colab import auth
auth.authenticate_user()

In [42]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'endless-bolt-246721'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [0]:
# Create a local file with data to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/
  
# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/to_upload.txt

In [13]:
import json
import os
import pprint
import re
import time
import tensorflow as tf


use_tpu = True #@param {type:"boolean"}
bucket = 'gs://xlnet_save_model' #@param {type:"string"}

assert bucket, 'Must specify an existing GCS bucket name'
print('Using bucket: {}'.format(bucket))

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

MODEL_DIR = 'gs://{}/{}'.format(bucket, time.strftime('tpuestimator-lstm/%Y-%m-%d-%H-%M-%S'))
print('Using model dir: {}'.format(MODEL_DIR))

from google.colab import auth
auth.authenticate_user()

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
  
  # Upload credentials to TPU.
  with tf.Session(TF_MASTER) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  pprint.pprint(session.list_devices())

Using bucket: gs://xlnet_save_model
Using model dir: gs://gs://xlnet_save_model/tpuestimator-lstm/2019-07-14-21-14-14


W0714 21:15:11.920043 140203918296960 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4222521744834103546),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3424450497210797554),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11032133522512063333),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16573041304298273466),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13257644097932464359),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11452646053239674523),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14013574602259663463),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 14998123271361789627),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5567499182534485086),
 _DeviceAttributes(/job:tpu_worker/replic

In [0]:
# Inspect the file we downloaded to /tmp
!cat /tmp/downloaded_from_gcs.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/colab')

## Setup
Install dependencies

In [3]:
! pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.4MB/s 


Download the pretrained XLNet model and unzip

In [14]:
# only needs to be done once
#! wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
! unzip cased_L-24_H-1024_A-16.zip /content 

Archive:  cased_L-24_H-1024_A-16.zip
caution: filename not matched:  /content


Download extract the imdb dataset - surpessing output

In [0]:
#! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar zxf aclImdb_v1.tar.gz

Git clone XLNet repo for access to run_classifier and the rest of the xlnet module

In [0]:
! git clone https://github.com/zihangdai/xlnet.git

In [0]:
import os
os.chdir('/content')

In [40]:
!mv /content/xlnet_cased_L-24_H-1024_A-16 gs://xlnet_save_model/xlnet_cased_L-24_H-1024_A-16
!ls

mv: cannot move '/content/xlnet_cased_L-24_H-1024_A-16' to 'gs://xlnet_save_model/xlnet_cased_L-24_H-1024_A-16': No such file or directory
adc.json  drive  exp  proc_data  sample_data  xlnet_cased_L-24_H-1024_A-16


## Define Variables
Define all the dirs: data, xlnet scripts & pretrained model. 
If you would like to save models then you can authenticate a GCP account and use that for the OUTPUT_DIR & CHECKPOINT_DIR - you will need a large amount storage to fix these models. 

Alternatively it is easy to integrate a google drive account, checkout this guide for [I/O in colab](https://colab.research.google.com/notebooks/io.ipynb) but rememeber these will take up a large amount of storage. 


In [0]:
SCRIPTS_DIR = 'drive/My\\ Drive/colab/xlnet' #@param {type:"string"}
DATA_DIR = 'drive/My\\ Drive/colab/aclImdb' #@param {type:"string"}
OUTPUT_DIR = 'gs://xlnet_save_model/proc_data/imdb' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'gs://xlnet_save_model/xlnet_cased_L-24_H-1024_A-16' #@param {type:"string"}
CHECKPOINT_DIR = 'gs://xlnet_save_model/exp/imdb' #@param {type:"string"}

## Run Model
This will set off the fine tuning of XLNet. There are a few things to note here:


1.   This script will train and evaluate the model
2.   This will store the results locally on colab and will be lost when you are disconnected from the runtime
3.   This uses the large version of the model (base not released presently)
4.   We are using a max seq length of 128 with a batch size of 8 please refer to the [README](https://github.com/zihangdai/xlnet#memory-issue-during-finetuning) for why this is.
5. This will take approx 4hrs to run on GPU.



In [10]:
train_command = "python xlnet/run_classifier.py \
  --do_train=True \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=8 \
  --eval_batch_size=8 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=4000 \
  --warmup_steps=500 \
  --save_steps=500 \
  --iterations=500"

! {train_command}


W0714 20:22:26.720641 140357121402752 deprecation_wrapper.py:119] From /content/drive/My Drive/colab/xlnet/model_utils.py:295: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0714 20:22:28.257902 140357121402752 deprecation_wrapper.py:119] From xlnet/run_classifier.py:855: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0714 20:22:28.258733 140357121402752 deprecation_wrapper.py:119] From xlnet/run_classifier.py:637: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0714 20:22:28.258885 140357121402752 deprecation_wrapper.py:119] From xlnet/run_classifier.py:637: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0714 20:22:28.259012 140357121402752 deprecation_wrapper.py:119] From xlnet/run_classifier.py:661: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0714 20:22:28.938753 140357121

In [0]:
train_command ="python drive/My\ Drive/colab/xlnet/run_classifier.py \
  --do_train=True \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --eval_batch_size=8 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=4000 \
  --warmup_steps=500 \
  --save_steps=500 \
  --iterations=500 \
  --use_tpu=True \
  --tpu='grpc://10.120.194.122:8470' \
  --num_core_per_host=8"

In [5]:
os.environ['COLAB_TPU_ADDR']

'10.120.194.122:8470'

In [56]:
! {train_command}

W0714 22:04:52.542717 139966757865344 deprecation_wrapper.py:119] From /content/drive/My Drive/colab/xlnet/model_utils.py:295: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0714 22:04:52.551923 139966757865344 deprecation_wrapper.py:119] From drive/My Drive/colab/xlnet/run_classifier.py:855: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0714 22:04:52.552931 139966757865344 deprecation_wrapper.py:119] From drive/My Drive/colab/xlnet/run_classifier.py:637: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0714 22:04:52.553115 139966757865344 deprecation_wrapper.py:119] From drive/My Drive/colab/xlnet/run_classifier.py:637: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0714 22:04:52.553268 139966757865344 deprecation_wrapper.py:119] From drive/My Drive/colab/xlnet/run_classifier.py:661: The name tf.gfile.Exists is

## Running & Results
These are the results that I got from running this experiment
### Params
*    --max_seq_length=128 \
*    --train_batch_size= 8 

### Times
*   Training: 1hr 11mins
*   Evaluation: 2.5hr

### Results
*  Most accurate model on final step
*  Accuracy: 0.92416, eval_loss: 0.31708


### Model

*   The trained model checkpoints can be found in 'exp/imdb'

